# Preambolo
Cerco di indurre i migliori alberi per le classificazioni 3 vs All, 4 vs All e maligni (1,2) vs benigni(3,4,5). Nel fare ciò cerco di combattere lo sbilanciamento delle classi passando class_weight = 'balanced' al costruttore dei decision tree, e utilizziando la score f1 "weighted"

10 iterazioni, per 4 vs all cerca di fare un holdout vicino a 70/30

Per ora la classe 5 la considero poco, è un po' indeterminata

f1-score--> meglio dell'accuratezza per classi non bilanciate
Cerco di aggirare lo sbolanciamento tra classi--> nell'allenamento degli alberi considero lo weight pesando più le classi meno rappresentate-->peso inversamente proporzionale al numero

Nel calcolo della score scelta media per le varie iterazioni visualizzo anche dev std.

Tutto ciò fatto sul Dataset NUOVO


## Moduli importati

In [1]:
import graphviz
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTENC, SMOTE
from sklearn import tree
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, f1_score, make_scorer, precision_score

## Importo dataset

In [2]:
#Ricordati di rimuovere la sheet vuota dal dataset, così da non dover specificare ogni volta Sheet7
#Ricorda di rimuovere le colonne coi nomi prima di caricare il notebook su GitHub!!!

df = pd.read_excel('DB PAROTIDE DEF_corretto.xlsx', sheet_name='Sheet7')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 27 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   NUM                             118 non-null    int64 
 1   COGNOME                         118 non-null    object
 2   NOME                            118 non-null    object
 3   DATA DI NASCITA                 118 non-null    object
 4   DATA RMN                        118 non-null    object
 5   Segni macro malignità           118 non-null    int64 
 6   ADC                             118 non-null    object
 7   TIC type                        118 non-null    object
 8   T2                              118 non-null    int64 
 9   Output Algoritmo                118 non-null    int64 
 10  FNAC                            118 non-null    int64 
 11  DATA FNAC                       117 non-null    object
 12  ESITO FNAC                      118 non-null    ob

## Nota 1
Nel dataset, tra i vari campi, ve ne è uno chamato "Escludere". I seguenti esperimenti, a meno che non sia diversamente specificato, non considerano i punti per i quali la feature "Escludere" è settata ad 1.
Inoltre, nei seguenti esperimenti, a meno che non sia diversamente specificato, utilizzo solo le seguenti features per l'induzione degli alberi: Segni macro malignità, ADC, TIC type, T2.
La label è sempre COD ISTO.
Per ora non considero l'età, o altri valori presenti nel DS

In [4]:
#Rimuovo i dati sensibili e non funzionali all'induzione degli alberi
#Rimuovo i pazienti con "Escludere" == 1
df = df.loc[df['ESCLUDERE'] == 0]
df = df.loc[:, ['Segni macro malignità', 'ADC', 'TIC type', 'T2', 'COD ISTO']]
df.head(10)


,Segni macro malignità,ADC,TIC type,T2,COD ISTO
0,0,"1,6",A,1,3
1,0,1,B,0,4
2,0,"1,2",C,0,3
4,0,"0,77",B,1,4
5,0,"0,8",C,1,1
7,0,"1,4",A,1,6
8,1,"0,7",C,1,1
10,0,"0,9",B,1,4
11,0,"0,7",B,1,4
12,0,"0,8",B,1,4


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107 entries, 0 to 114
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Segni macro malignità  107 non-null    int64 
 1   ADC                    107 non-null    object
 2   TIC type               107 non-null    object
 3   T2                     107 non-null    int64 
 4   COD ISTO               107 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 5.0+ KB


## NB
Noto che nella colonna dell'output dell'algoritmo ci sono alcuni valori a 0 e che alcuni sembrano non combaciare con i riusultati dell'algoritmo di classificazione che avevo implementato tempo fa.
Ai fini dell'induzione degli alberi questo non è rilevante ma sicuramente è il caso che controlli ulteriormenti questa cosa.


In [6]:
#Converto ADC da string a float
df['ADC'] = df['ADC'].apply(lambda s: str(s).replace(',', '.')).astype(float)
df.head(5)

,Segni macro malignità,ADC,TIC type,T2,COD ISTO
0,0,1.60,A,1,3
1,0,1.00,B,0,4
2,0,1.20,C,0,3
4,0,0.77,B,1,4
5,0,0.80,C,1,1


## Samples e labels

In [7]:
X_not_encoded = df.iloc[:,:-1].copy()
X = pd.get_dummies(X_not_encoded, columns = ['TIC type'])
y = df['COD ISTO'].copy()
y.value_counts()

3    53
4    27
1    12
5     6
2     5
6     4
Name: COD ISTO, dtype: int64

## NOTA 2
Nel dataset non ci sono più punti con tic type D. Inoltre qua la classe 6 ricompare. Per ora la considero

## Funzione per l'esperimento

In [8]:
#qui non divido in test set e training set ma faccio direttamente gridsearch
#non prevedo augmentation con smote
#inoltre qua prevedo 4 splits, cioè un rapporto test set / training set = 0.33 (0.25/0.75)
#Così so che in ogni cv il test set stratificato contiene almeno un rtappresentante pe rogni classe
#da notare che se si sceglie di usare f1 come score viene applicata la versione weighted che dà più peso agli errori
#se commessi su classi minoritarie

def GS(X, y, target_one, tuned_parameters, 
                scoring_func = accuracy_score, n_splits= 4,
                random_state_tree = None,
                n_it = 10, filename = 'albero'):
    
    scores = []
    models = []
    if scoring_func is f1_score:
        scoring = make_scorer(scoring_func, average = 'weighted')
    else:
        scoring = make_scorer(scoring_func)
        
    cv = StratifiedKFold(n_splits = n_splits, shuffle = True, random_state = None)
    
    y = [1 if x in target_one else 0 for x in y]
    
    for n in range(n_it):
        
        print('### ITERATION %d ###\n\n' %(n+1))
        print('# Tuning hyper-parameters #\n')

        
        albero = tree.DecisionTreeClassifier(random_state = random_state_tree)
        

        clf = GridSearchCV(albero, tuned_parameters, scoring = scoring, cv = cv, refit = True)
        clf.fit(X, y)
        models.append(clf.best_estimator_)

        params = clf.best_params_
        score = clf.best_score_
 
 
        print("Parameters set found:\n")
        print(params)
        print()
        print("Mean cross-validated score of the estimator: %0.3f\n" %(score))
        print('-'*100)
        
        scores.append(score)
        
    print('\n\n# Final Mean score: %0.3f\n# Std dev: %0.3f' %(np.mean(scores), np.std(scores)))
    
    best_model = models[np.argmax(scores)]
    
    data = tree.export_graphviz(best_model, out_file=None, 
                      feature_names=X.columns,  
                      class_names=['0','1'],  
                      filled=True, rounded=True,  
                      special_characters=True) 
    graph = graphviz.Source(data) 
    graph.render('GS-newDS-'+filename)
    

In [9]:
seed = 42 

tuned_parameters = {'criterion': ['gini', 'entropy'],
                    'max_depth': list(range(2,11)),     
                    'min_samples_leaf': list(range(2,11)),
                    'class_weight': ['balanced', None]}

In [10]:
GS(X,y, [3], tuned_parameters, random_state_tree = seed, scoring_func = accuracy_score, filename = '3-acc')

### ITERATION 1 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 3}

Mean cross-validated score of the estimator: 0.897

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.879

----------------------------------------------------------------------------------------------------
### ITERATION 3 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 3}

Mean cross-validated score of the estimator: 0.906

----------------------------------------------------------------------------------------------------
### ITERATION 4 ###


# Tuning hyper-parameters #

Para

In [11]:
GS(X,y, [3], tuned_parameters, random_state_tree = seed, scoring_func = f1_score, filename = '3-f1')

### ITERATION 1 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.887

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 3}

Mean cross-validated score of the estimator: 0.888

----------------------------------------------------------------------------------------------------
### ITERATION 3 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 6}

Mean cross-validated score of the estimator: 0.897

----------------------------------------------------------------------------------------------------
### ITERATION 4 ###


# Tuning hyper-parameters #

Para

In [12]:
GS(X,y, [4], tuned_parameters, random_state_tree = seed, scoring_func = accuracy_score, filename = '4-acc')

### ITERATION 1 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 1.000

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 1.000

----------------------------------------------------------------------------------------------------
### ITERATION 3 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.981

----------------------------------------------------------------------------------------------------
### ITERATION 4 ###


# Tuning hyper-parameters #

In [13]:
GS(X,y, [4], tuned_parameters, random_state_tree = seed, scoring_func = f1_score, filename = '4-f1')

### ITERATION 1 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 1.000

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 1.000

----------------------------------------------------------------------------------------------------
### ITERATION 3 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.982

----------------------------------------------------------------------------------------------------
### ITERATION 4 ###


# Tuning hyper-parameters #

In [14]:
GS(X,y, [1,2], tuned_parameters, random_state_tree = seed, scoring_func = accuracy_score, filename = '12-acc')

### ITERATION 1 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.963

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.963

----------------------------------------------------------------------------------------------------
### ITERATION 3 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.963

----------------------------------------------------------------------------------------------------
### ITERATION 4 ###


# Tuning hyper-parameters #

Parameters

In [15]:
GS(X,y, [1,2], tuned_parameters, random_state_tree = seed, scoring_func = f1_score, filename = '12-f1')

### ITERATION 1 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.960

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.963

----------------------------------------------------------------------------------------------------
### ITERATION 3 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.963

----------------------------------------------------------------------------------------------------
### ITERATION 4 ###


# Tuning hyper-parameters #

P

In [ ]:
#rifaccio esattamente gli stessi esperimenti, diminuendo di 1 le fold

In [16]:
GS(X,y, [3], tuned_parameters, random_state_tree = seed, scoring_func = accuracy_score,
   filename = '3-acc-3splits', n_splits = 3)

### ITERATION 1 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.878

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.898

----------------------------------------------------------------------------------------------------
### ITERATION 3 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.897

----------------------------------------------------------------------------------------------------
### ITERATION 4 ###


# Tuning hyper-parameters #

In [17]:
GS(X,y, [3], tuned_parameters, random_state_tree = seed, scoring_func = f1_score,
   filename = '3-f1-3splits', n_splits = 3)

### ITERATION 1 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 3}

Mean cross-validated score of the estimator: 0.897

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.888

----------------------------------------------------------------------------------------------------
### ITERATION 3 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 2, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.887

----------------------------------------------------------------------------------------------------
### ITERATION 4 ###


# Tuning hyper-parameters #

P

In [18]:
GS(X,y, [4], tuned_parameters, random_state_tree = seed, scoring_func = accuracy_score,
   filename = '4-acc-3splits', n_splits = 3)

### ITERATION 1 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.981

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.981

----------------------------------------------------------------------------------------------------
### ITERATION 3 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.981

----------------------------------------------------------------------------------------------------
### ITERATION 4 ###


# Tuning hyper-parameters #

In [19]:
GS(X,y, [4], tuned_parameters, random_state_tree = seed, scoring_func = f1_score,
   filename = '4-f1-3splits', n_splits = 3)

### ITERATION 1 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 1.000

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.982

----------------------------------------------------------------------------------------------------
### ITERATION 3 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 1.000

----------------------------------------------------------------------------------------------------
### ITERATION 4 ###


# Tuning hyper-parameters #

In [20]:
GS(X,y, [1,2], tuned_parameters, random_state_tree = seed, scoring_func = accuracy_score,
   filename = '12-acc-3splits', n_splits = 3)

### ITERATION 1 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.972

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 3}

Mean cross-validated score of the estimator: 0.972

----------------------------------------------------------------------------------------------------
### ITERATION 3 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.963

----------------------------------------------------------------------------------------------------
### ITERATION 4 ###


# Tuning hyper-parameters #

Parameters se

In [21]:
GS(X,y, [1,2], tuned_parameters, random_state_tree = seed, scoring_func = f1_score,
   filename = '12-f1-3splits', n_splits = 3)

### ITERATION 1 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.961

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.944

----------------------------------------------------------------------------------------------------
### ITERATION 3 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.960

----------------------------------------------------------------------------------------------------
### ITERATION 4 ###


# Tuning hyper-parameters #

Paramet

In [33]:
#A questo punto provo a valutare accuratezza e f1score weighted dell'albero che compie una
#classificazione multiclasse

def GS_multiclass(X, y, tuned_parameters, 
                scoring_func = accuracy_score, n_splits= 4,
                random_state_tree = None,
                n_it = 10, filename = 'albero'):
    
    scores = []
    models = []
    if scoring_func is f1_score:
        scoring = make_scorer(scoring_func, average = 'weighted')
    else:
        scoring = make_scorer(scoring_func)
        
    cv = StratifiedKFold(n_splits = n_splits, shuffle = True, random_state = None)
    
    for n in range(n_it):
        
        print('### ITERATION %d ###\n\n' %(n+1))
        print('# Tuning hyper-parameters #\n')

        
        albero = tree.DecisionTreeClassifier(random_state = random_state_tree)
        

        clf = GridSearchCV(albero, tuned_parameters, scoring = scoring, cv = cv, refit = True)
        clf.fit(X, y)
        models.append(clf.best_estimator_)

        params = clf.best_params_
        score = clf.best_score_
 
 
        print("Parameters set found:\n")
        print(params)
        print()
        print("Mean cross-validated score of the estimator: %0.3f\n" %(score))
        print('-'*100)
        
        scores.append(score)
        
    print('\n\n# Final Mean score: %0.3f\n# Std dev: %0.3f' %(np.mean(scores), np.std(scores)))
    
    best_model = models[np.argmax(scores)]
    
    class_names = []
    for i in y.unique():
        class_names.append(str(i))
    class_names.sort()
    
    data = tree.export_graphviz(best_model, out_file=None, 
                      feature_names=X.columns,  
                      class_names=class_names,  
                      filled=True, rounded=True,  
                      special_characters=True) 
    graph = graphviz.Source(data) 
    graph.render('GS-newDS-multi'+filename)

In [34]:
#partiamo usando 4 split
GS_multiclass(X,y, tuned_parameters, random_state_tree = seed, scoring_func = accuracy_score,
   filename = '-acc-4splits', n_splits = 4)

### ITERATION 1 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.832

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.850

----------------------------------------------------------------------------------------------------
### ITERATION 3 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.832

----------------------------------------------------------------------------------------------------
### ITERATION 4 ###


# Tuning hyper-parameters #

Parameters set f

In [35]:
GS_multiclass(X,y, tuned_parameters, random_state_tree = seed, scoring_func = f1_score,
   filename = '-f1-4splits', n_splits = 4)

### ITERATION 1 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.802

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.815

----------------------------------------------------------------------------------------------------
### ITERATION 3 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.786

----------------------------------------------------------------------------------------------------
### ITERATION 4 ###


# Tuning hyper-parameters #

In [36]:
#e ripeto per 3 splits
GS_multiclass(X,y, tuned_parameters, random_state_tree = seed, scoring_func = accuracy_score,
   filename = '-acc-3splits', n_splits = 3)

### ITERATION 1 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 4, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.841

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 3, 'min_samples_leaf': 5}

Mean cross-validated score of the estimator: 0.832

----------------------------------------------------------------------------------------------------
### ITERATION 3 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 2, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.832

----------------------------------------------------------------------------------------------------
### ITERATION 4 ###


# Tuning hyper-parameters #

Paramet

In [37]:
GS_multiclass(X,y, tuned_parameters, random_state_tree = seed, scoring_func = f1_score,
   filename = '-f1-3splits', n_splits = 3)

### ITERATION 1 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.791

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.814

----------------------------------------------------------------------------------------------------
### ITERATION 3 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 4, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.791

----------------------------------------------------------------------------------------------------
### ITERATION 4 ###


# Tuning hyper-parameter